In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import keras 
print(keras.__version__)

In [3]:
#### STEP-1 Loading all necessary libraries 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2, random, time, shutil, csv
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
np.random.seed(42)
%matplotlib inline 
import json
import os
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

In [4]:
#Data Paths
train_dir = '/kaggle/input/dog-breed-identification/train/'
test_dir = '/kaggle/input/dog-breed-identification/test/'
#Count/Print train and test samples.

In [5]:
#Read train labels.
labels_dataframe = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
#Read sample_submission file to be modified by pridected labels.
sample_df = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
#Incpect labels_dataframe.
labels_dataframe.head(5)
# here sample df is the format of data to be submitted into kaggle 


In [6]:
labels_dataframe.shape

In [7]:
# little reminder of working with file and folder with os 
print(f"number of pic in test_dir {len(os.listdir(test_dir))}")
print(f"number of pic in train dir {len(os.listdir(train_dir))}")

so here the size of labels_dataframe and the size of train directory is the same 
and in train directory every file have unique name and in labels_dataframe we can see the label of that picture and labels_dataframe and train directory both are in ordered manned 

In [8]:
# lets crate the  number of unique labels we have available in our dataset 
dog_breeds = sorted(list(set(labels_dataframe.breed)))
n_classes = len(dog_breeds)
print(n_classes)
print(dog_breeds[:10])


# here we have 120 dog breeeds to classify 

In [9]:
# so these dog breeds are string so we need to change them into some number for classification
#Map each label string to an integer label.
class_to_num = dict(zip(dog_breeds, range(n_classes)))
class_to_num

In [10]:
(train_dir+labels_dataframe.id+'.jpg')[0]
# this is how we will access the files from our training data 

In [11]:
cv2.imread((train_dir+labels_dataframe.id+'.jpg')[0]).shape
# size of image 

In [12]:
plt.imshow(cv2.imread((train_dir+labels_dataframe.id+'.jpg')[0]))

In [13]:
labels_dataframe['file_name'] = labels_dataframe['id'].apply(lambda x:train_dir+f"{x}.jpg")

In [14]:
labels_dataframe.head()

In [15]:
# now we need to map our breed category according the class_to_num 
labels_dataframe['breed'] = labels_dataframe.breed.map(class_to_num)


In [16]:
labels_dataframe.head()

In [17]:
y = to_categorical(labels_dataframe.breed)
# encoded our y variable to pass in our model

#### Uptill now we have created our dataframe with file names and categories 

#### Lets Design model architecture for feature extraction


In [18]:
from keras.applications.resnet_v2 import ResNet50V2 , preprocess_input as resnet_preprocess
from keras.applications.densenet import DenseNet121, preprocess_input as densenet_preprocess
from keras.layers.merge import concatenate

input_shape = (331,331,3)
input_layer = Input(shape=input_shape)


#first extractor inception_resnet
preprocessor_resnet = Lambda(resnet_preprocess)(input_layer)
inception_resnet = ResNet50V2(weights = 'imagenet',
                                     include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_resnet)

preprocessor_densenet = Lambda(densenet_preprocess)(input_layer)
densenet = DenseNet121(weights = 'imagenet',
                                     include_top = False,input_shape = input_shape,pooling ='avg')(preprocessor_densenet)


merge = concatenate([inception_resnet,densenet])
model = Model(inputs = input_layer, outputs = merge)

In [19]:
model.summary()

In [20]:
model.save('feature_extractor.h5')

In [21]:
loaded_model = keras.models.load_model('./feature_extractor.h5')

In [22]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes = True)

In [23]:
model.output.shape

In [24]:
len(model.trainable_weights)

#### Feature Extraction by usinng pretrained models 

In [25]:
# for feature_extraction dataframe must have to contain file_name and  breed columns
def feature_extractor(df):
    img_size = (331,331,3)
    data_size = len(df)
    batch_size = 20
    X = np.zeros([data_size,3072], dtype=np.uint8)
#     y = np.zeros([data_size,120], dtype=np.uint8)
    datagen = ImageDataGenerator() # here we dont need to do any image augementaion because we are prediction features 
    generator = datagen.flow_from_dataframe(df,
    x_col = 'file_name', class_mode = None, 
    batch_size=20, shuffle = False,target_size = (img_size[:2]),color_mode = 'rgb')
    i = 0
    
    for input_batch in tqdm(generator):
        input_batch = model.predict(input_batch)
        X[i * batch_size : (i + 1) * batch_size] = input_batch
        i += 1
        if i * batch_size >= data_size:
            break
    return X
 

In [26]:
X = feature_extractor(labels_dataframe)


In [27]:
X.shape

In [28]:
X


#### Defining some model callbacks for our training

In [29]:
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
#Prepare call backs
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
checkpoint = ModelCheckpoint('/kaggle/working/checkpoint',
                             monitor = 'val_loss',mode = 'min',save_best_only= True)
lr = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.5,patience = 3,min_lr = 0.00001)
my_callback=[EarlyStop_callback,checkpoint]

### Lets create a model for training 


In [30]:
dnn = keras.models.Sequential([
    InputLayer(X.shape[1:]),
    Dropout(0.7),
    Dense(n_classes, activation='softmax')
])

dnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Train simple DNN on extracted features.
h = dnn.fit(X , y,
            batch_size=128,
            epochs=60,
            validation_split=0.1 ,
           callbacks = my_callback)

#### Plot the result

In [31]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

ax1.plot(h.history['loss'],color = 'b',label = 'loss')
ax1.plot(h.history['val_loss'],color = 'r',label = 'val_loss')
ax1.set_xticks(np.arange(1, 60, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))
ax1.legend(['loss','val_loss'],shadow = True)


ax2.plot(h.history['accuracy'],color = 'green',label = 'accuracy')
ax2.plot(h.history['val_accuracy'],color = 'red',label = 'val_accuracy')
ax2.legend(['accuracy','val_accuracy'],shadow = True)
# ax2.set_xticks(np.arange(1, 60, 1))
# ax2.set_yticks(np.arange(0, 60, 0.1))
plt.show()

In [32]:
# saving the model
from keras.models import load_model
dnn.save('/kaggle/working/dogbreed.h5')

In [33]:
import pickle
with open('dog_breeds_category.pickle', 'wb') as handle:
    pickle.dump(dog_breeds, handle, protocol=pickle.HIGHEST_PROTOCOL)

### its time for prediction 


In [34]:
test_data = []
ids = []
for pic in os.listdir(test_dir):
    ids.append(pic.split('.')[0])
    test_data.append(test_dir+pic)

In [35]:
test_dataframe = pd.DataFrame({'file_name':test_data})
# we are converting into a dataframe beacause our feature extractor funtion only support dataframe

In [36]:
test_features = feature_extractor(test_dataframe)


In [37]:
y_pred = dnn.predict(test_features)

In [38]:
def get_key(val): 
    for key, value in class_to_num.items(): 
        if val == value: 
            return key 
pred_codes = np.argmax(y_pred, axis = 1)
predictions = []
for i in pred_codes:
    predictions.append(get_key(i))

In [39]:
test_dataframe['breed'] = predictions

In [40]:
np.set_printoptions(suppress=True)

In [41]:
pd.set_option('display.float_format', lambda x: '%.10f' % x)

#### creating submission  file for the competetion

In [42]:
submission = pd.DataFrame(y_pred, columns = dog_breeds)
submission['id'] = ids


In [43]:
submission.set_index('id')

In [44]:
submission.to_csv('sumbmission.csv', index = False)

### Plotting some images 

In [45]:
plt.figure(figsize=(6,6))

for index , data in test_dataframe[:10].iterrows():
    img = data['file_name']
    label = data['breed']
    img = cv2.imread(img)
#     plt.subplot(2,5, index+1)
    plt.imshow(img)
    plt.xlabel(label,fontsize = (15))
    plt.tight_layout()
    plt.show()